### Imports

In [1]:
### Standard imports
import os
import random
import numpy as np
import pandas as pd
import nibabel as nib

from glob import glob
from os.path import join
from tqdm import tqdm_notebook as tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
os.environ['KMP_DUPLICATE_LIB_OK']='True'

### Grey masks

In [4]:
masks = glob("/Volumes/hd_4tb/masks/plip/*")
masks = [{
    "code": os.path.basename(mask).split("_")[0],
    "data": nib.load(mask).get_data(),
} for mask in masks]

def _in_mask(masks, x, y, z):
    result = dict()
    for mask in masks:
        code = mask["code"]
        data = mask["data"]
        result["in_%s" % code] = int(bool(data[x, y, z]))
    return result

def _mean_activation(masks, fmri, grey, t, label):
    activations = dict()
    for mask in masks:
        code, data = mask["code"], mask["data"]
        region = np.multiply(data, fmri[:, :, :, t])
        activations["mean_%s_%s" % (code, label)] = np.mean( np.multiply(grey, region) )
    return activations

def _mask_info(masks, fmri, grey, coord):
    x, y, z, t = coord
    info = _in_mask(masks, x, y, z)
    for timepoint, label in [(t-1, "prev"), (t+1, "next")]:
        info = {**info, **_mean_activation(masks, fmri, grey, timepoint, label)}
    info["x"] = x
    info["y"] = y
    info["z"] = z
    info["t"] = t
    return info

print(_in_mask(masks, 49, 66, 33))

{'in_001716': 1, 'in_013136': 0, 'in_073720': 0, 'in_125909': 0, 'in_131520': 0, 'in_156999': 0, 'in_176064': 0, 'in_182355': 0, 'in_216192': 0, 'in_275836': 0, 'in_291082': 0, 'in_414207': 0, 'in_426426': 0, 'in_447647': 0, 'in_450767': 0, 'in_477522': 0, 'in_525931': 0, 'in_532911': 0, 'in_533294': 0, 'in_534482': 0, 'in_707656': 0, 'in_752691': 0, 'in_779062': 0, 'in_791103': 0, 'in_831650': 0, 'in_877949': 0, 'in_911981': 0, 'in_954861': 0}


### fMRI

In [5]:
def _load_volume(fmri, x, y, z, t):
    volume = nii_input(fmri[:, :, :, t], x, y, z)
    return np.array(volume)

def _load(filepath):
    return nib.as_closest_canonical(nib.load(filepath))

def _get_data(filepath):
    image = _load(filepath)
    return image.get_data()

def _time_map(session):
    _map = {
        "s1": "000", "s2": "2MO", "s3": "6MO", "s4": "12MO", "s5": "24MO"
    }
    return _map[session]

def _get(row, item):
    return row[item]

def _fmri_path(row):
    _input = "/Volumes/hd_4tb/raw"
    project = _get(row, "project")
    subject = _get(row, "subject")
    time_session = _get(row, "time_session")
    task = _get(row, "task")
    return join(_input, project, _time_map(time_session), subject, task)

def nii_input(data, x, y, z, r = 4):
    return data[
        x - r : x + r + 1,
        y - r : y + r + 1,
        z - r : z + r + 1
    ]

### Onsets

In [6]:
def _onset_time(onsets, curr_time):
    onset = onsets[onsets["ons"] < curr_time]
    onset = onset.sort_values("ons", ascending=False)
    if len(onset) > 0:
        _time = onset.iloc[0].ons
        return curr_time - _time
    return False

def _stim_time(df, stimuli, curr_time):
    stim = df[df.category == stimuli]
    return _onset_time(stim, curr_time)

def _keypress_times(df, button, curr_time):
    keys = df[(
        (df["category"] == "keypress") &
        (df["stimulus"] == button)
    )]
    return _onset_time(keys, curr_time)

def last_onset(onset_df, task, curr_time, max_time=1000):
    task_stimuli = {
        "gonogo":       ["Go", "NoGo"],
        "conscious":    ["Anger", "Disgust", "Fear", "Happy", "Neutral", "Sad"],
        "nonconscious": ["Anger", "Disgust", "Fear", "Happy", "Neutral", "Sad"],
        "workingmemSB": ["Baseline", "NonTarget", "Target"],
        "workingmemMB": ["Baseline", "NonTarget", "Target"],
    }
    all_stimuli  = set(np.concatenate(list(task_stimuli.values())))
    onset_timing = {stimuli: max_time for stimuli in all_stimuli}
    for button in ["1", "6"]:
        onset_timing[button] = _keypress_times(onset_df, button, curr_time)
        
    for stimuli in task_stimuli[task]:
        _time = _stim_time(onset_df, stimuli, curr_time)
        if _time:
            onset_timing[stimuli] = _time
    return pd.DataFrame(onset_timing, index=[1])

### Load model

In [7]:
# model = _load_model()
# keras.utils.plot_model(model, show_shapes=True)

### Setup

In [8]:
df                = pd.read_csv("/Volumes/hd_4tb/project/model_input.csv")
available_volumes = np.load("available_volumes.npy")
train_cols        = [c for c in df.columns if c.startswith("is_")] + ["age"]

training_path = "/Volumes/hd_4tb/results/training"
batch_size  = 128

### Train

For working with multiple inputs/outputs see [here](https://www.tensorflow.org/guide/keras/functional#models_with_multiple_inputs_and_outputs).  NN advice [here](https://stats.stackexchange.com/questions/352036/what-should-i-do-when-my-neural-network-doesnt-learn?noredirect=1&lq=1)

In [10]:
def cartesian(data, timepoints):
    ret = list()
    for x, y, z in data:
        for t in timepoints:
            ret.append((x, y, z, t))
    return ret

for i, row in df.iterrows():
    if i < 453:
        continue
    print(_get(row, "project"), _get(row, "subject"), _get(row, "time_session"), _get(row, "task"))
    TR = 2 if _get(row, "is_mb") == 0 else 0.71
    task = _get(row, "task")
    if task == "workingmemMB":
        continue

    fmri_path = _fmri_path(row)
    fmri      = _get_data(join(fmri_path, "normalized.nii.gz"))
    grey      = _get_data(join(fmri_path, "..", "structural", "gm_probseg.nii.gz"))
    onset_df  = pd.read_csv(join(fmri_path, "onsets.csv"))
    info = pd.concat([pd.DataFrame(row[train_cols]).T] * batch_size).reset_index(drop=True)
    
    batch = {"prev": list(), "next": list()}
    bold_signal, mask_rows = list(), list()
    training_voxels = cartesian( available_volumes, np.array(range(1, fmri.shape[3]-1)) )
    training_index = random.sample(range(len(training_voxels)), batch_size*10)

    for j, index in tqdm(enumerate(training_index), total=batch_size*10):
        x, y, z, t = training_voxels[index]
        onsets = last_onset(onset_df, task, TR * t, max_time=1000)
        onsets = pd.concat([onsets] * batch_size).reset_index(drop=True)
        
        mask_rows.append( _mask_info(masks, fmri, grey, (x, y, z, t))  )
        grey_data = nii_input(grey, x, y, z)
        for name, timepoint in [("prev", t-1), ("next", t+1)]:
            batch[name].append(
                np.stack((_load_volume(fmri, x, y, z, timepoint), grey_data), axis=3)
            )
        bold_signal.append(fmri[x, y, z, t])
        
        ### Fit to data
        if (j + 1) % batch_size == 0:
            input_folder = join(training_path, "%04d/%02d" % (i, j // batch_size))
            if not os.path.isdir(input_folder):
                os.makedirs(input_folder)
            batch["info"] = pd.concat([
                onsets, info, pd.DataFrame(mask_rows)
            ], axis=1)
            batch["prev"], batch["next"] = np.array(batch["prev"]), np.array(batch["next"])
            preds = np.array(bold_signal)
            np.save(join(input_folder, "prev.npy"), batch["prev"])
            np.save(join(input_folder, "next.npy"), batch["next"])
            np.save(join(input_folder, "info.npy"), batch["info"])
            np.save(join(input_folder, "pred.npy"), preds)
            batch = {"prev": list(), "next": list()}
            bold_signal, mask_rows = list(), list()

connmdd conn209 s1 gonogo


connmdd conn209 s1 nonconscious


connmdd conn209 s1 workingmemMB
connmdd conn211 s1 conscious


connmdd conn211 s1 gonogo


connmdd conn211 s1 nonconscious


connmdd conn211 s1 workingmemMB
engage la13012 s1 conscious


engage la13012 s1 gonogo


engage la13012 s1 nonconscious


engage la13272 s1 conscious


KeyboardInterrupt: 